In [2]:
import string

#read in corpora
used_nr_corpus_sentences = 30 #688670 lines
corpus_path = "./Paralel Corpus/"
filename_e = "BU_en.txt"
filename_f = "BU_tr.txt"

e_file = open(corpus_path + filename_e, "r", encoding='utf-8-sig')
e_lines = e_file.readlines()
f_file = open(corpus_path + filename_f, "r", encoding='utf-8-sig')
f_lines = f_file.readlines()

def tokenize(lines):
    lines = [line.rstrip() for line in lines[:used_nr_corpus_sentences]] #remove \n
    words = [line.split() for line in lines] #split by words
    unpunctuated = str.maketrans('', '', string.punctuation) #remove punctuation & make Lowercase
    lower = [[word.translate(unpunctuated).lower() for word in sentence] for sentence in words]
    return lower

f = tokenize(f_lines)
e = tokenize(e_lines)

In [16]:
import numpy as np

max_steps = 20
training_split = 0.9

def split_dataset(dataset, training_split):
    np.random.shuffle(dataset)
    nr_train_samples = int(training_split*len(dataset))
    return dataset[:nr_train_samples], dataset[nr_train_samples:]

e_train, e_test = split_dataset(e, training_split)
f_train, f_test = split_dataset(f, training_split)
e_train = [['the','house'],['the','book'],['a','book']]
f_train = [['das','Haus'],['das','Buch'],['ein','Buch']]

e_words = set([item for sublist in e_train for item in sublist])
f_words = set([item for sublist in f_train for item in sublist])
    
#test if the two sets are equivalent
def is_converged(t, last_t):
    if ( t == last_t): print( t, last_t)
    return t == last_t

In [20]:
import copy
#learning translation probabilitity distributions from sentence-aligned parallel text
#expectation maximization algorithm
#Input: set of sentence pairs (e,f) 
#Output: translation prob. t(e|f)
def EM_IBM_Model_1 (e_set,f_set):
    #sets of distinct words
    e_words = set([item for sublist in e_set for item in sublist])
    f_words = set([item for sublist in f_set for item in sublist])
    #initialize t(e|f) with uniform distribution
    t = {}#{key: {key2: 1/len(e_words) for key2 in f_words} for key in e_words}
    for k in range (0, len(e_set)):
        e = e_set[k]
        f = f_set[k]
        for e_j in e:
            for f_i in f:
                t[(e_j,f_i)] = t.get((e_j,f_i),1/len(e_words))    
    last_t = {0:1} #to fail first comparison
    step = 0
    #iterate until convergence
    while not(is_converged(t, last_t)) and step < max_steps:
        #initialize
        count = {e: {f: 0 for f in f_words} for e in e_words}
        total = {f: 0 for f in f_words}
        for k in range (0, len(e_set)):
            e = e_set[k]
            f = f_set[k]
            #compute normalization
            #s_total = {key: 0 for key in e_words}
            s_total = {}
            for e_j in e:
                s_total[e_j] = 0
                for f_i in f:
                    s_total[e_j] += t[(e_j,f_i)]
            #collect counts
            for f_i in f:
                for e_j in e:
                    count[e_j][f_i] += t[(e_j,f_i)] / s_total[e_j]
                    total[f_i] += t[(e_j,f_i)] / s_total[e_j]
        #estimate probabilities
        last_t = copy.deepcopy(t)
        for f_i in f_words:
            for e_j in e_words:
                if (e_j,f_i) in t:
                    t[(e_j,f_i)] = count[e_j][f_i] / total[f_i] 
        step += 1
        print(last_t)
        print('---')
        #print('count ',count)
        #print('stotal ',s_total)
        #print('total',total)
        #print('last_t',last_t)
    return t
print(e_train)
t = EM_IBM_Model_1(e_train,f_train)
print(t)
#how to iterate lengths to initialize a,
#is sum over t ok?, why wrong values!

[['the', 'house'], ['the', 'book'], ['a', 'book']]
{('the', 'das'): 0.25, ('the', 'Haus'): 0.25, ('house', 'das'): 0.25, ('house', 'Haus'): 0.25, ('the', 'Buch'): 0.25, ('book', 'das'): 0.25, ('book', 'Buch'): 0.25, ('a', 'ein'): 0.25, ('a', 'Buch'): 0.25, ('book', 'ein'): 0.25}
---
{('the', 'das'): 0.5, ('the', 'Haus'): 0.5, ('house', 'das'): 0.25, ('house', 'Haus'): 0.5, ('the', 'Buch'): 0.25, ('book', 'das'): 0.25, ('book', 'Buch'): 0.5, ('a', 'ein'): 0.5, ('a', 'Buch'): 0.25, ('book', 'ein'): 0.5}
---
{('the', 'das'): 0.6363636363636364, ('the', 'Haus'): 0.4285714285714286, ('house', 'das'): 0.18181818181818182, ('house', 'Haus'): 0.5714285714285715, ('the', 'Buch'): 0.18181818181818182, ('book', 'das'): 0.18181818181818182, ('book', 'Buch'): 0.6363636363636364, ('a', 'ein'): 0.5714285714285715, ('a', 'Buch'): 0.18181818181818182, ('book', 'ein'): 0.4285714285714286}
---
{('the', 'das'): 0.7478974515333995, ('the', 'Haus'): 0.34661354581673304, ('house', 'das'): 0.13126000457351933

In [217]:
#learning translation probabilitity distributions from sentence-aligned parallel text
#expectation maximization algorithm
#Input: set of sentence pairs (e,f) 
#Output: translation prob. t (lexical translation) and a (alignment)
def EM_IBM_Model_2 (e_set,f_set):
#TODO jump parameterisation for better performance???
    #initialize t(e|f) with IBM Model 1
    t = EM_IBM_Model_1(e_set,f_set)
    #initialize a(i|j, l_e, l_f)
    e_lengths = [len(e) for e in e_set]
    f_lengths = [len(f) for f in f_set]
    for l_e in e_lengths:
        for l_f in f_lengths:
            for i in range(0,l_f):
                for j in range(0,l_e):
                    a[(i,j,l_e,l_f)] = 1/(l_f + 1)
    #sets of distinct words
    e_words = list(set([item for sublist in e_set for item in sublist]))
    f_words = list(set([item for sublist in f_set for item in sublist]))
    last_t = {} #to fail first comparison
    step = 0
    #iterate until convergence
    while ((not is_converged(t, last_t)) | step < max_steps):
        #initialize
        count = {e: {f: 1/len(e_words) for f in f_words} for e in e_words}
        total = {f: 0 for f in f_words}
        total_a = {}
        count_a = {}
        for l_e in e_lengths:
            for l_f in f_lengths:
                for j in range(0,l_e):
                    total_a[(j,l_e,l_f)] = 0
                    for i in range(0,l_f):
                        count_a[(i,j,l_e,l_f)] = 0
        for k in range (0, len(e_set)):
            e = e_set[k]
            f = f_set[k]
            l_e = len(e)
            l_f = len(f)
            #compute normalization
            s_total = {key: 0 for key in e_words}
            for j in range(0, l_e):
                for i in range(0, l_f):
                    e_j = e[j]
                    f_i = f[i]
                    s_total[e_j] += t[e_j][f_i] * a[(i,j,l_e,l_f)]
            #collect counts
            for j in range(0, l_e):
                for i in range(0, l_f):
                    e_j = e[j]
                    f_i = f[i]
                    c = t[e_j][f_i] * a[(i,j,l_e,l_f)] / s_total[e_j]
                    count[e_j][f_i] += c
                    total[f_i] += c
                    count_a[(i,j,l_e,l_f)] += c
                    total_a[(j,l_e,l_f)] += c
        #estimate probabilities
        last_t = t
        for f_i in f_words:
            for e_j in e_words:
                t[e_j][f_i] = count[e_j][f_i] / total[f_i]
        for l_e in e_lengths:
            for l_f in f_lengths:
                for i in range(0,l_f):
                    for j in range(0,l_e):
                        a[(i,j,l_e,l_f)] = count_a[(i,j,l_e,l_f)] / total_a[(j,l_e,l_f)]    
        step += 1
        #print(t)
    return t, a

t, a = EM_IBM_Model_2(e_train,f_train)
print(t)

{'the': {'ein': 0.25, 'das': 1.125, 'Buch': 0.125, 'Haus': 0.25}, 'house': {'ein': 0.25, 'das': 0.125, 'Buch': 0.125, 'Haus': 1.25}, 'book': {'ein': 0.25, 'das': 0.125, 'Buch': 1.125, 'Haus': 0.25}, 'a': {'ein': 1.25, 'das': 0.125, 'Buch': 0.125, 'Haus': 0.25}}


In [4]:
#Testing
def prob_e_given_f (e, f, epsilon, t):
    t_sum = np.sum([list(i.values()) for i in t.values()])
    return ( epsilon / len(f)**len(e) ) * t_sum
    #without +1 for NONE
    #here problem: sum over t and t also contains unused values!

for i in range(0,len(f_test)):
    f = f_test[i]
    e = e_test[i]
    #decoding
    #calculate p(e|f)
    #print(e, prob_e_given_f(e, f, 0.001, t))
    
f = ['das','Buch']
e = ['the','book']
print('p(',e,'|',f,') =',prob_e_given_f(e, f, 1, t))
e = ['kgefw','fek']
print('p(',e,'|',f,') =',prob_e_given_f(e, f, 1, t))

p( ['the', 'book'] | ['das', 'Buch'] ) = 1.7040281502999115
p( ['kgefw', 'fek'] | ['das', 'Buch'] ) = 1.7040281502999115


In [ ]:
#Phrase-based model

#Sentence Alignments in both directions with Viterbi algorithm using IBM Model 2
#While it is true that during the EM training fractional counts are
#collected over a probability distribution of possible alignments, one
#word alignment sticks out: the most probable alignment, also called the
#Viterbi alignment
#>  output the Viterbi alignment of the last iteration
#> run IBM model training in both directions

#for each sentence combine the two alignments using the word alignment algorithm
#>merged by taking the intersection or the union of alignment points of each alignment
#The algorithm starts with the intersection of the alignments. In
#the growing step, neighboring alignment points that are in the union but
#not in the intersection are added. In the final step, alignment points for
#words that are still unaligned are added.

#call that for every sentence pair!
def GROW_DIAG():
    while (last_a != a):
        for english word e = 0 ... en
            for foreign word f = 0 ... fn
                if ( e aligned with f )
                    for each neighboring point ( e-new, f-new ):
                        if ( ( e-new not aligned or f-new not aligned ) and ( e-new, f-new ) in union( e2f, f2e ) )
                            add alignment point ( e-new, f-new )

def FINAL(a):
    for english word e-new = 0 ... en
        for foreign word f-new = 0 ... fn
            if ( ( e-new not aligned or f-new not aligned ) and ( e-new, f-new ) in union( e2f, f2e) 
                add alignment point ( e-new, f-new )
                
def GROW_DIAG_FINAL(e2f,f2e):
    neighboring = [(-1,0),(0,-1),(1,0),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    alignment = intersect(e2f,f2e)
    GROW_DIAG()
    FINAL(e2f)
    FINAL(f2e)
#phrase extraction algorithm Use a suitable threshold for maximum phrase length and 
#estimate the phrase translation probabilities. 

#test the model using the test data. 
#for a source sentence, you can generate some example target sentences (word sequences) 
#and phrases “manually”; i.e. you will do the decoding process manually. 
#That is, for a source sentence, generate a possible target sentence,
#divide both sentences into phrases as you wish, and assume that you know the phrase
#correspondences. Then, calculate the probability of the target sentence given the source
#sentence using the p(f|e)*p LM (e) equation (i.e. using the standard model with three
#components). 

#You can train a simple bigram language model and a reordering model.